## 13.2 Dagger-Hilt - Podstawy - ROOM

W tej aplikacji zaimplementujemy architekturę **MVVM** z wykorzystaniem `Dagger-Hilt` oraz lokalną bazą `ROOM`. Rozpocznijmy od dodania zależności.

In [ ]:
// build.gradle(Project)
id 'com.google.dagger.hilt.android' version '2.44' apply false

In [ ]:
plugins {
    id 'com.android.application'
    id 'org.jetbrains.kotlin.android'
    id 'kotlin-kapt'
    id 'dagger.hilt.android.plugin'
    id 'kotlin-android'
}

dependencies {

    implementation "com.google.dagger:hilt-android:2.44"
    kapt "com.google.dagger:hilt-android-compiler:2.44"
    kapt "androidx.hilt:hilt-compiler:1.0.0"


    // activity dla kotlin KTX + ViewModel
    implementation "androidx.activity:activity-ktx:1.5.1"

    // ROOM
    implementation("androidx.room:room-ktx:2.4.3")
    kapt("androidx.room:room-compiler:2.4.3")

    // ViewModel
    implementation 'androidx.lifecycle:lifecycle-viewmodel-ktx:2.5.1'
    // LiveData
    implementation "androidx.lifecycle:lifecycle-livedata-ktx:2.5.1"
}

Zdefiniujmyn model danych.

In [ ]:
@Entity(tableName = "student")
data class Student(
    @PrimaryKey(autoGenerate = true) var id: Int = 0,
    val name: String)

Następnie dodajmy klasę zwracającą instancję bazy - tutaj musimy przekazać `Context` w metodzie `getDatabase`.

In [ ]:
@Database(entities = [Student::class], version = 1)
abstract class AppDatabase() : RoomDatabase() {

    abstract fun appDao(): AppDao

    companion object{
        @Volatile private var INSTANCE: AppDatabase? = null

        fun getDatabase(context: Context): AppDatabase {
            return INSTANCE ?: synchronized(this) {
                val instance = Room.databaseBuilder(
                    context.applicationContext,
                    AppDatabase::class.java,
                    "student_database_kotlin"
                ).build().also { INSTANCE = it }
                instance
            }
        }
    }
}

Kolejnym elementem bazy jest `Dao` - tutaj dodamy dwie metody do odczytania wszystkich danych oraz dodania pojedynczego elementu.

In [ ]:
@Dao
interface AppDao {
    @Insert(onConflict = OnConflictStrategy.IGNORE)
    suspend fun addItem(student: Student)

    @Query("SELECT * FROM student")
    fun readAllData(): LiveData<List<Student>>
}

W metodzie dostarczającej musimy przekazać `Application` jako `Context` do utworzenia `ROOM`. Biblioteka `Hilt` automatycznie wykorzysta instancję klasy `AppAplication`, którą musimy zdefiniować jako context - ponieważ rozszerza ona klasę `Application`, czas żecia `ROOM` jest tożsamy z czasem życia samej aplikacji.

In [ ]:
@HiltAndroidApp
class AppApplication : Application() {}

In [ ]:
<manifest xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools">

    <application
        android:name=".AppApplication"
...

Dodajmy moduł aplikacji - jak w poprzednim przykładzie, wykorzystamy `SingletonComponent`

In [ ]:
@Module
@InstallIn(SingletonComponent::class)
object AppModule {}

Do modułu dodajmy metodę `getAppDB` dostarczającą instancję bazy danych

In [ ]:
@Singleton
@Provides
fun getAppDB(app: Application): AppDatabase{
    return AppDatabase.getDatabase(app)
}

Jak widzimy `getAppDB` przyjmuje instancję `Application` - `Hilt` automatycznie dostarcza instancję klasy oznaczonej jako `@HiltAndroidApp`.

Drugą metodą jest `getDao` dostarczająca instancję obiektu implementującego interfejs `AppDao`.

In [ ]:
@Singleton
@Provides
fun getDao(db: AppDatabase): AppDao{
    return db.appDao()
}

Następnym elementem **MVVM** będzie repozytorium, po raz kolejny posłużymy się interfejsem oraz klasą implementującą ten interfejs.

In [ ]:
interface AppRepository {
    val readAllData: LiveData<List<Student>>
    suspend fun insert(student: Student)
}

In [ ]:
class AppRepositoryImpl @Inject constructor (private val appDao: AppDao) : AppRepository {
    override val readAllData: LiveData<List<Student>> = appDao.readAllData()
    override suspend fun insert(student: Student) = appDao.addItem(student)
}

Do repozytorium dodajemy instancję `AppDao` stosując wstrzyknięcie przez konstruktor.

Powróćmy do `AppModule` i dodajmy metodę dostarczającą repozytorium.

In [ ]:
@Singleton
@Provides
fun provideAppRepository(appDao: AppDao) : AppRepository {
    return AppRepositoryImpl(appDao)
}

Ostatnim elementem **MVVM** jest `ViewModel`. Przyjmuje dwa argumenty - `Application` oraz `AppRepository` - które dostarczymy stosując wstrzyknięcie przez konstruktor.

In [ ]:
@HiltViewModel
class AppViewModel @Inject constructor(
    app: Application,
    private  val repository: AppRepository
) : AndroidViewModel(app) {

Kontekst aplikacji musimy przekazać do konstruktora klasy `AndroidViewModel`, jest to jedyne miejsce gdzie jest on wymagany, więc nie musimy tworzyć pola - stąd brak `val`/`var`.

Dodajmy dwie wcześniej opisane metody.

In [ ]:
val readAllData: LiveData<List<Student>> = repository.readAllData

fun insert(student: Student) = viewModelScope.launch {
    repository.insert(student)

Główną aktywność oznaczamy jako `@AndroidEntryPoint`. Dodajmy `ViewModel`.

In [ ]:
private val viewModel: AppViewModel by viewModels()

W metodzie `onCreate` dodajmy kilka elementów do bazy oraz obserwator.

In [ ]:
viewModel.apply {
    insert(Student(0, "Rafa"))
    insert(Student(0, "Maciej"))
    insert(Student(0, "Kuba"))

    readAllData.observe(this@MainActivity) { student ->
        val content = StringBuilder()
        student.forEach {
            content
                .append("id: ").append(it.id).append("\n")
                .append("Name: ").append(it.name).append("\n\n")
        }
        textView.text = content
    }
}